Calls C program with parameters that are given in the arrays

In [80]:
from subprocess import call, check_output
import subprocess
import numpy as np

NNodes = [3200]
NEdges = [6400]
NOpinions = [320]
Phi100 = list(np.arange(0, 101, 1))
MAX_ITER = [100000000000]
ITER_step = [1000]
filename = "data1"
red = 1000

#for i in range(red):
#    for NN in NNodes:
#        for NE in NEdges:
#            for NO in NOpinions:
#                for Phi in Phi100:
#                    for M_ITER in MAX_ITER:
#                        for step in ITER_step:
#                            print(i)
#                            print(check_output(["./a.out", str(NN), str(NE), str(NO), str(Phi), str(M_ITER), str(step), filename]))

Now with multiprocessing:

In [ ]:
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing
import itertools


results2 = []
#As above just with a filename dependent on the worker to avoid corruptions. Will create a file for each thread
def RunCProgram(NN, NE, NO, phi, M_Iter, step):
    filename ="multitest"
    #print(multiprocessing.current_process()._identity)
    filename = filename + str( multiprocessing.current_process()._identity[0])
    #print(filename)
    if M_Iter < step:
        print(M_Iter, step)
    for x in range(1000):
        try:
            check_output(["./a.out", str(NN), str(NE), str(NO), str(phi), str(M_Iter), str(step), filename])
        except subprocess.CalledProcessError as e:
            raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
    return





if __name__ == '__main__':
    NThreads = multiprocessing.cpu_count() - 1
    param = []
    param.append(NNodes)
    param.append(NEdges)
    param.append(NOpinions)
    param.append(Phi100)
    param.append(MAX_ITER)
    param.append(ITER_step)
    InputArray = list(itertools.product(*param))
    print(InputArray[0])
    with Pool(processes=NThreads) as pool:         # start 4 worker processes
        pool.starmap(RunCProgram, InputArray)       # prints "[0, 1, 4,..., 81]"

(3200, 6400, 320, 0, 100000000000, 1000)


[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
